##This notebook is for creating and testing tensorflowlite model from tensorflow model created from my first notebook

In [1]:
!pip install keras-image-helper

In [2]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
     |████████████████████████████████| 2.2 MB 8.3 MB/s 


In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [4]:
import tensorflow.lite as tflite

In [5]:
from PIL import Image

In [6]:
model = keras.models.load_model('/content/drive/MyDrive/xception_v4_1_01_0.567.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('xception_v4_1_01_0.567.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpj9js813l/assets


INFO:tensorflow:Assets written to: /tmp/tmpj9js813l/assets


In [ ]:
import tflite_runtime.interpreter as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='/content/drive/MyDrive/xception_v4_1_01_0.567.tflite')
interpreter.allocate_tensors()

In [ ]:
input_index = interpreter.get_input_details()[0]['index']

In [ ]:
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
def preprocess_input(diir):
  with Image.open(diir) as img:
    img = img.resize((250, 250), Image.NEAREST)
  x = np.array(img, dtype='float32')
  X = np.array([x])
  X = X/255
  return X

In [ ]:
def preprocessIMG(fu):
  img = load_img(fullname, target_size=(250, 250))
  X = np.array(img)
  X = np.array([X])
  X = X/255
  return X

In [ ]:
pic = preprocess_input('/content/drive/MyDrive/610968546.jpg')


In [ ]:
interpreter.set_tensor(input_index, pic)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
preds

array([[7.5975990e-01, 1.8870127e-01, 3.6846798e-02, 7.2378032e-05,
        1.4619679e-02]], dtype=float32)

In [ ]:
preds[0].tolist()

[0.7597599029541016,
 0.18870127201080322,
 0.03684679791331291,
 7.23780322005041e-05,
 0.014619679190218449]

In [ ]:
disease_class_list = ['Cassava Bacterial Blight (CBB)',
 'Cassava Brown Streak Disease (CBSD)',
 'Cassava Green Mottle (CGM)',
 'Cassava Mosaic Disease (CMD)',
 'Healthy']

In [ ]:
disease_class_list[np.argmax(preds)]

'Cassava Bacterial Blight (CBB)'

In [ ]:
def predict(diir):
  with Image.open(diir) as img:
    img = img.resize((250, 250), Image.NEAREST)
  x = np.array(img, dtype='float32')
  X = np.array([x])
  X = X/255
  interpreter.set_tensor(input_index, X)
  interpreter.invoke()
  preds = interpreter.get_tensor(output_index)
  float_predictions = preds[0].tolist()
  return dict(zip(disease_class_list, float_predictions)),disease_class_list[np.argmax(preds)]

In [ ]:
predict('/content/drive/MyDrive/610968546.jpg')

({'Cassava Bacterial Blight (CBB)': 0.7597599029541016,
  'Cassava Brown Streak Disease (CBSD)': 0.18870127201080322,
  'Cassava Green Mottle (CGM)': 0.03684679791331291,
  'Cassava Mosaic Disease (CMD)': 7.23780322005041e-05,
  'Healthy': 0.014619679190218449},
 'Cassava Bacterial Blight (CBB)')